In [48]:
#------------------------------------------
# Lab 2 - GTI770
#------------------------------------------


# Import des libraries necessaires
from skimage import io, novice
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris

#For Keras MLP
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import normalize

#http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

#http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split
from sklearn.model_selection import train_test_split, KFold

import numpy as np
import operator 

#https://docs.python.org/2/library/csv.html
import csv

%matplotlib inline


#--------------------------------------------------------------------------------------------------------
#
# -------------------------------------MAIN PROGRAM------------------------------------------------------


#
#
#
#
#------------------Section Définition de fonctions-----------
#
#
#
#

# On a besoin de cette fonction car la conversion de nparray laisse toutes les données en np_str et on veux float
def convertStringsToFloats(array):
    convertedArray = []
    for element in array:
        convertedArray.append(float(element))
    return convertedArray

#Séparation des données en test/train
def floatMapTrainValidTest(data, labels):

    trainFeatures, validFeatures, trainLabels, validLabels = train_test_split(data, labels, test_size=0.30)
    validFeatures, testFeatures, validLabels, testLabels = train_test_split(validFeatures, validLabels, test_size=0.33)
    
    floatDataMap = {"trainFeatures" : [], 
                   "trainLabels" : [], 
                   "validFeatures" : [],
                   "validLabels" : [],
                   "testFeatures" : [], 
                   "testLabels" : []
                   }
    
    for row in trainFeatures:
        floatDataMap["trainFeatures"].append(convertStringsToFloats(row))
        
    for row in validFeatures:
        floatDataMap["validFeatures"].append(convertStringsToFloats(row))
        
    for row in testFeatures:
        floatDataMap["testFeatures"].append(convertStringsToFloats(row))
        
    for row in trainLabels:
        floatDataMap["trainLabels"].append(row)
        
    for row in validLabels:
        floatDataMap["validLabels"].append(row)
        
    for row in testLabels:
        floatDataMap["testLabels"].append(row)
        
    return floatDataMap


#Conversion des labels de validation de format bizarres
def arrangeLabels(labels):
    newLabels = []
    for label in labels:
        newLabels.append(label[0])
    return newLabels

#Printing of graphs
def printGraph(x, y, title):

    plt.plot(x, y)

    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

     
def crossValidation(trainLabels, trainFeatures, validationLabels, validationFeatures, testLabels, testFeatures, case):
    
    print()
    print(case)
    
    dataFeatures = trainFeatures + validationFeatures + testFeatures
    dataLabels = trainLabels + validationLabels + testLabels
    
    accuracyScoreList = []
    f1ScoreList = []
    
    kf = KFold(n_splits=10)
    for train_index, test_index in kf.split(dataFeatures):
        
        
        newTrainFeatures = []
        newTrainLabels = []
        newTestFeatures = []
        newTestLabels = []
        
        
        for index in train_index:
            newTrainFeatures.append(dataFeatures[index])
            newTrainLabels.append(dataLabels[index])
            
        for index in test_index:
            newTestFeatures.append(dataFeatures[index])
            newTestLabels.append(dataLabels[index])
        
        #Add iteratively testing model function logic in here
    
    print("Cross-Val Accuracy: ")   
    print(np.mean(accuracyScoreList))
    print("Cross-Val F1: ")
    print(np.mean(f1ScoreList))
        
# À modifier pour votre environmement - Galaxy
galaxyFeatureVectorFolderLocation = 'C:/Users/turco/gti770/'

# Lecture de fichier -> Utiliser après l'extraction des données du TP01
# Ici on a besoin du CSV 
galaxyVectorFid = open(galaxyFeatureVectorFolderLocation + 'features.csv', 'r') # option r veut dire read
galaxyVectorTxt = galaxyVectorFid.readlines() # Cette ligne permet de lire tout le fichier 
galaxyVectorFid.close() 

galaxyFeatureVector=[]


#
#
#
#
#------------------Section Extraction de primitives (CSV) (Galaxie)-----------
#
#
#
#

for line in galaxyVectorTxt:
    columns = line[:-1]
    columns = columns.split(',')
    galaxyFeatureVector.append(columns)
    
    

#
#
#
#
#------------------Section Manipulation---------------------
#
#
#
#

#Manipulations des données pour la séparation en apprentissage, validation et test

# Explication de 'splicing': 
# iris.data[:, :3] veut dire qu'on utilise la totalite des "rows" et seulement 3 colonnes [:, 3]
# pour utiliser seulement 3 "rows" et la totalite des colonnes on utilise [:3, :]
galaxyFeatureVector = np.array(galaxyFeatureVector)

spiralFeatures = []
spiralLabels = []

smoothFeatures = []
smoothLabels = []

#Séparation par étiquettes

#Séparation des galaxies
for row in galaxyFeatureVector:
    if (row[0] == "spiral"):
        spiralFeatures.append(row[1:])
        spiralLabels.append(0)
    else:
        smoothFeatures.append(row[1:])
        smoothLabels.append(1)
        

#Séparation de train/valid/test

smoothData = {"trainFeatures" : [], 
           "trainLabels" : [], 
           "validFeatures" : [],
           "validLabels" : [],
           "testFeatures" : [], 
           "testLabels" : []
           }


spiralData = {"trainFeatures" : [], 
           "trainLabels" : [], 
           "validFeatures" : [],
           "validLabels" : [],
           "testFeatures" : [], 
           "testLabels" : []
           }

#Séparation galaxies avec one hot encoding pour les étiquettes!
smoothData = floatMapTrainValidTest(smoothFeatures, smoothLabels)
spiralData = floatMapTrainValidTest(spiralFeatures, spiralLabels)

trainFeatures = normalize(np.asarray(smoothData["trainFeatures"] + spiralData["trainFeatures"]))
trainLabels = to_categorical(smoothData["trainLabels"] + spiralData["trainLabels"])

testFeatures = normalize(np.asarray(smoothData["testFeatures"] + spiralData["testFeatures"]))
testLabels = to_categorical(smoothData["testLabels"] + spiralData["testLabels"])

validationFeatures = normalize(np.asarray(smoothData["validFeatures"] + spiralData["validFeatures"]))
validationLabels = to_categorical(smoothData["validLabels"] + spiralData["validLabels"])


# unique_elements, counts_elements = np.unique(trainLabels, return_counts=True)

# print(unique_elements)
# print(counts_elements)

#
#
#
#
#------------------Section Réseau de neuronnes----------------
#
#
#
#

model = Sequential()
model.add(Dense(units=100, activation='sigmoid', input_dim=77))
model.add(Dense(units=100, activation='sigmoid'))
model.add(Dense(units=2, activation='sigmoid'))

#Learning rate en paramètre içi avec l'optimization SGD
sgd = keras.optimizers.SGD(lr=0.0005)

model.compile(optimizer=sgd, loss='mse', metrics=['accuracy'])

#Utilise: tensorboard --logdir=./logs/keras_nn --host=127.0.0.1 Appèle cette fonction 
tb_callback = keras.callbacks.TensorBoard(log_dir='./logs/keras_nn')

model.fit(trainFeatures, trainLabels, epochs=25, batch_size=100, callbacks=[tb_callback])

print(model.predict(testFeatures))

model.summary()



#
#
#
#
#---------------------------Section SVM-----------------------
#
#
#
#







#--------------------------------------------------------------------------------------------------------
#
# -------------------------------------END PROGRAM-------------------------------------------------------

[0. 1.]
[0. 1.]
[11835 11835]
Epoch 1/25
11835/11835 [==============================] - 0s 21us/step - loss: 0.3281 - acc: 0.5190
Epoch 2/25
11835/11835 [==============================] - 0s 12us/step - loss: 0.3203 - acc: 0.5190
Epoch 3/25
11835/11835 [==============================] - 0s 12us/step - loss: 0.3127 - acc: 0.5190
Epoch 4/25
11835/11835 [==============================] - 0s 12us/step - loss: 0.3055 - acc: 0.5190
Epoch 5/25
11835/11835 [==============================] - 0s 13us/step - loss: 0.2987 - acc: 0.5190
Epoch 6/25
11835/11835 [==============================] - 0s 12us/step - loss: 0.2924 - acc: 0.5190
Epoch 7/25
11835/11835 [==============================] - 0s 12us/step - loss: 0.2866 - acc: 0.5190
Epoch 8/25
11835/11835 [==============================] - 0s 14us/step - loss: 0.2813 - acc: 0.5190
Epoch 9/25
11835/11835 [==============================] - 0s 13us/step - loss: 0.2766 - acc: 0.5190
Epoch 10/25
11835/11835 [==============================] - 0s 12us/ste